In [ ]:
import pandas as pd
#import MetaTrader5 as mt5
import talib as ta
from bb_setup_1 import setup
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pontos = 3
rate_stop = 1
rate_tp = 2
spread = 0.00005

df_total = None

for timeframe in [5,15]:
    for var_bb in [2.5,3]:
        for pontos in [0.0050, 0.0070,0.0100]:
            for rate_tp in [1.5,2,2.5,3,5]:
                
                ativo = f'GBPUSD{timeframe}'
                colnames = ['Data','Open','High','Low','Close','Vol']
                df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep = '\t')
                df = df[df['Data'] > '2022-06-01']

                df_acao = setup(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

                qtd_operacoes = df_acao.shape[0]
                taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
                operacoes_certas = qtd_operacoes * taxa_acerto
                operacoes_erradas = qtd_operacoes - operacoes_certas
                pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

                pontos_liquido = pontos_bruto - spread * qtd_operacoes


                df_new = pd.DataFrame([[ativo,var_bb,rate_tp,rate_stop,pontos,qtd_operacoes,taxa_acerto,pontos_bruto,pontos_liquido]],
                            columns = ['ativo','var_bb','rate_tp','rate_stop','pontos','qtd_operacoes','taxa_acerto','pontos_bruto','pontos_liquido'])

                df_total = pd.concat([df_total,df_new])

                print(df_total.to_string())

In [ ]:
#df_total.to_csv('result_bb_setup_1_eurusd.csv')
df_total = pd.read_csv('result_bb_setup_1_eurusd.csv')

In [ ]:
df_1 = df_total[df_total['ativo'] == 'EURUSD1']
df_5 = df_total[df_total['ativo'] == 'EURUSD5']
df_15 = df_total[df_total['ativo'] == 'EURUSD15']

In [ ]:
df_1.sort_values(by = 'pontos_liquido', ascending = False).head(10)

In [ ]:
df_5.sort_values(by = 'pontos_liquido', ascending = False).head(20)

In [ ]:
df_15.sort_values(by = 'pontos_liquido', ascending = False).head(50)

In [ ]:
df_15.loc[[85]]

In [ ]:
timeframe = 15
for index,row in df_15.sort_values(by = 'pontos_liquido', ascending = False).loc[[115]].iterrows():

    pontos = row['pontos']
    rate_tp = row['rate_tp']
    var_bb = row['var_bb']

    ativo = f'EURUSD{timeframe}'
    colnames = ['Data','Open','High','Low','Close','Vol']
    df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep='\t')
    #df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
    df = df[df['Data'] > '2022-01-01']

    df_acao = setup(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

    qtd_operacoes = df_acao.shape[0]
    taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes


    print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

    print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

    df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
    df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
    df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

    df_new_mes = df_acao[['Mes','resultado_binario']]
    df_new_hora = df_acao[['Hora_h','resultado_binario']]

    def calc_pontos(x):
        qtd_operacoes = x['qtd_operacoes']
        taxa_acerto = x['taxa_acerto']
        operacoes_certas = qtd_operacoes * taxa_acerto
        operacoes_erradas = qtd_operacoes - operacoes_certas
        pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

        pontos_liquido = pontos_bruto - spread * qtd_operacoes

        return pontos_liquido

    print('Agrupado por Hora')
    df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
    print(df_new_hora.to_string())

    print('Agrupado por Mes')
    df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

    df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
    print(df_new_mes.to_string())
    print('')


In [ ]:
ativo = f'EURUSD{15}_new'
colnames = ['Data','Open','High','Low','Close','Vol']
#df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, sep='\t')
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)
df = df[df['Data'] > '2022-01-01']

var_bb = 2
rate_tp = 1.5
pontos = 0.0003

df_acao = setup(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

In [ ]:
df_acao[['Data','acao','stop','tp','resultado_binario']].tail(20)

In [ ]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]